In [4]:
from swarm import Swarm, Agent
from swarm.types import Result
from dotenv import load_dotenv
from typing import Dict, List, Tuple
import os
from openai import OpenAI

load_dotenv()

openai_client = OpenAI()

# Define the instructions for the Supply Agent
dispatch_agent_instructions = """
You are a Dispatch Agent responsible dispatching announcements to the corresponding locations.

You will be a list of elements in the format:
(Origin, Destination, Category, [Items])
This is already in the context variables, simply call the send_dispatch_multiple function.

Your task is to call the function twice for each element, once for the origin and once for the destination.
The items are already provided to you in the context variables.
"""

dispatchs = { "dispatchs": [('Seafood Market C', 'Community Kitchen Z', 'seafood', ['salmon']), ('Bakery B', 'Homeless Shelter Y', 'grains', ['bread']), ('Grocery Store A', 'Soup Kitchen X', 'fruits', ['strawberry']), ('Grocery Store A', 'Homeless Shelter Y', 'dairy', ['milk']), ('Bakery B', 'Community Kitchen Z', 'baked goods', ['muffins']), ('Seafood Market C', 'Soup Kitchen X', 'seafood', ['shrimp'])]}

locations = {
    "locations": {
        # Suppliers
        "Grocery Store A": {
            "surplus": ["fruits", "dairy"],
            "surplus_mapping": {
                "fruits": ["strawberry", "apple"],
                "dairy": ["milk", "cheese"]
            },
            "data": {
                "lat": 40.712776,
                "lon": -74.005974,
                "address": "123 Main St, New York, NY 10001"
            }
        },
        "Bakery B": {
            "surplus": ["grains", "baked goods"],
            "surplus_mapping": {
                "grains": ["bread", "rice"],
                "baked goods": ["muffins", "cookies"]
            },
            "data": {
                "lat": 42.652580,
                "lon": -73.756232,
                "address": "456 Country Rd, Albany, NY 12207"
            }
        },
        "Seafood Market C": {
            "surplus": ["seafood"],
            "surplus_mapping": {
                "seafood": ["salmon", "shrimp"]
            },
            "data": {
                "lat": 42.886448,
                "lon": -78.878372,
                "address": "789 Market St, Buffalo, NY 14202"
            }
        },
        
        # Demanders
        "Soup Kitchen X": {
            "demand": ["fruits", "seafood"],
            "data": {
                "lat": 40.717054,
                "lon": -73.984472,
                "address": "321 Charity Ave, New York, NY 10002"
            }
        },
        "Homeless Shelter Y": {
            "demand": ["dairy", "grains"],
            "data": {
                "lat": 42.652580,
                "lon": -73.756232,
                "address": "654 Support St, Albany, NY 12205"
            }
        },
        "Community Kitchen Z": {
            "demand": ["baked goods", "seafood"],
            "data": {
                "lat": 42.886448,
                "lon": -78.878372,
                "address": "987 Hope Rd, Buffalo, NY 14203"
            }
        }
    }
}


def send_dispatch_multiple(context_variables):
    """
    This function sends dispatch messages to the origin and destination for each dispatch in the context variables.

    Args:
        None

    Returns:
        str: A message indicating that all dispatch messages have been sent.
    """
    for dispatch in context_variables["dispatchs"]:
        send_dispatch(context_variables, dispatch)
        
    return "Finished dispatching all items."
        
        
def send_dispatch(context_variables, dispatch):
    origin, destination, category, items = dispatch
    
    sending_prompt = f"""
    Write a friendly text message to the {origin} location, notifying them that {destination} will be coming to pick up {items} of {category} from them. They should be prepared to package the items neatly.
    """
    
    receiving_prompt = f"""
    Write a friendly text message to the {destination} location, notifying them the {origin} location has extra food of {category} that they can use. They have {items} available. They should be prepared to send someone to pick it up.
    
    Origin location: {context_variables["locations"][origin]["data"]["address"]}
    """
    
    sending_completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": sending_prompt}],
        temperature=0.8
    )
    
    print(sending_completion.choices[0].message.content)
    
    receiving_completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": receiving_prompt}],
        temperature=0.8
    )   
    
    print(receiving_completion.choices[0].message.content)
    
    return True

In [5]:
dispatch_agent = Agent(
    name="Dispatch Agent",
    instructions=dispatch_agent_instructions,
    functions=[send_dispatch_multiple]
    
)



In [6]:
# Initialize the Swarm client
client = Swarm()
# Simulate user messages indicating surplus
messages = [
    {"role": "user", "content": "Help me dispatch the surplus items to the demand locations."}
]

# Define the function call context
def run_dispatch_agent():
    # Run the Supply Agent with the provided messages and additional arguments
    response = client.run(
        agent=dispatch_agent, messages=messages, context_variables= {
            **locations,
            **dispatchs
        }
    )
    return response

# Execute the Logistics Agent
response = run_dispatch_agent()

# Output the response
print("Final Agent:", response.agent.name)
print("Context Variables:", response.context_variables)
print("\nMessages:")
for msg in response.messages:
    print(f"{msg['role']}: {msg['content']}")

Hi there! 😊 Just wanted to give you a heads up that Community Kitchen Z will be coming by to pick up some salmon from your location. If you could have it packaged neatly for us, that would be fantastic! Thank you so much for your help! 🐟👍
Hi Community Kitchen Z! 😊 Just wanted to let you know that Seafood Market C has some extra salmon available that you might be able to use. If you’re interested, please send someone over to pick it up. The address is 789 Market St, Buffalo, NY 14202. Let me know if you have any questions! Thanks! 🐟✨
Hi Bakery B! 😊 Just a quick note to let you know that Homeless Shelter Y will be coming by to pick up some bread made from grains. If you could please have everything packaged neatly, that would be wonderful! Thank you so much for your support! 🍞❤️
Hi there! 😊 This is a friendly note to let you know that Bakery B has some extra food available, specifically bread. They’re located at 456 Country Rd, Albany, NY 12207. If you’d like to send someone to pick it u